In [1]:
import os

import pprint as pp
from IPython.display import IFrame

import rangekeeper as rk

In [2]:
# Authenticate with Speckle
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN')) # Note if you run this notebook yourself, you'll need to set this environment variable



 SpeckleClient( server: https://speckle.xyz, authenticated: True )


In [3]:
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)

In [4]:
# Load the design scenario
model = speckle.get_commit(stream_id=stream_id) # Providing no commit_id defaults to the latest commit
model

Base(id: 2bd634ec952d0fab78d5e48f4ceb467d, speckle_type: Base, totalChildrenCount: 2894)

In [5]:
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)

In [6]:
# Let's identify the root members of the scenario:
roots = model.get_dynamic_member_names()
roots

['@scenario', '@context']

In [7]:
IFrame("https://speckle.xyz/streams/{0}/objects/{1}".format(stream_id, model['@scenario']['id']), width='100%', height=800)

In [8]:
property_name = 'type'
IFrame('https://speckle.xyz/streams/{0}/commits/{1}?filter=%7B%22propertyInfoKey%22%3A%22{2}%22%7D'.format(
    stream_id,
    commit_id,
    property_name), width='100%', height=800)

In [9]:
# Return any Speckle Objects in the '@scenario' trunk that have 'entityId's:
parsed = rk.api.Speckle.parse(base=model['@scenario'])

# (Recursively) Convert the Speckle Objects into Rangekeeper Entities:
scenario = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='scenario',
    type='scenario')

# (We can check that it is an Assembly:)
isinstance(scenario, rk.graph.Assembly)

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


True

In [10]:
# Get the "BuildingA" Assembly:
buildingA = [entity for (entityId, entity) in scenario.get_entities().items() if entity['name'] == 'buildingA'][0]
print('buildingA: {0}'.format(buildingA))

buildingA: Assembly: buildingA
Type: building
Members: ['type', 'name']
Entities: [('529ce7bc-addf-4bad-8ec5-cbecbd73d4f1', {'entity': Assembly: buildingA (Type: building)}), ('987c5866-03b0-4948-962e-4870684f481b', {'entity': Assembly: buildingAparking (Type: space)}), ('7dc6bf46-805f-4990-bcd2-d0fe1de4dd47', {'entity': Assembly: buildingAretail (Type: space)}), ('549a988b-c3bc-42d4-a6a3-a8dc694fa770', {'entity': Assembly: buildingAresidential (Type: space)}), ('3cf2bf71-61fa-4a61-b163-6265c299376d', {'entity': Assembly: buildingAretail (Type: space)}), ('bfc1c272-665f-432d-a7b3-a4c955556e7f', {'entity': Assembly: buildingAoffice (Type: space)}), ('55963d84-702d-423d-be05-c8bb6e88f517', {'entity': Entity: buildingAcores (Type: utilities)})]
Relationships: [('529ce7bc-addf-4bad-8ec5-cbecbd73d4f1', '987c5866-03b0-4948-962e-4870684f481b', 'spatiallyContains'), ('529ce7bc-addf-4bad-8ec5-cbecbd73d4f1', '7dc6bf46-805f-4990-bcd2-d0fe1de4dd47', 'spatiallyContains'), ('529ce7bc-addf-4bad-8ec5-

In [11]:
# Get all relatives of BuildingA where BuildingA is the source of
# a 'spatiallyContains' relationship:
buildingA_containment = buildingA.get_relatives(
    outgoing=True,
    relationship_type='spatiallyContains',
    assembly=scenario)
print('buildingA Containment: \n {0}\n'.format(buildingA_containment))

buildingA Containment: 
 [Assembly: buildingAparking (Type: space), Assembly: buildingAretail (Type: space), Assembly: buildingAresidential (Type: space), Assembly: buildingAretail (Type: space), Assembly: buildingAoffice (Type: space), Entity: buildingAcores (Type: utilities)]



In [12]:
buildingAresi = [entity for entity in buildingA_containment if entity['name'] == 'buildingAresidential'][0]
print('buildingAresidential: {0}'.format(buildingAresi))

buildingAresidential: Assembly: buildingAresidential
Type: space
Members: ['renderMaterial', 'use', '@displayValue', 'volume', 'type', 'name']
Entities: [('549a988b-c3bc-42d4-a6a3-a8dc694fa770', {'entity': Assembly: buildingAresidential (Type: space)}), ('e7b3aec0-5003-448b-a0e3-4c2dbb605f85', {'entity': Entity: buildingAresidentialFloor0 (Type: floor)}), ('9c13f0be-9991-4d9f-b41b-a0e962bdeb89', {'entity': Entity: buildingAresidentialFloor0 (Type: floor)}), ('e88ff464-a453-45bb-93d8-5f983d417912', {'entity': Entity: buildingAresidentialFloor1 (Type: floor)}), ('8ce80d06-69f6-404a-b0fd-a728eb0dfed6', {'entity': Entity: buildingAresidentialFloor2 (Type: floor)}), ('bb0a0d86-31c1-4780-b81f-8d0e5a5d83be', {'entity': Entity: buildingAresidentialFloor3 (Type: floor)})]
Relationships: [('549a988b-c3bc-42d4-a6a3-a8dc694fa770', 'e7b3aec0-5003-448b-a0e3-4c2dbb605f85', 'spatiallyContains'), ('549a988b-c3bc-42d4-a6a3-a8dc694fa770', '9c13f0be-9991-4d9f-b41b-a0e962bdeb89', 'spatiallyContains'), ('54

In [13]:
scenario.plot(name='assets/design_scenario')
IFrame(src="./design_scenario.html", width='100%', height=800)

assets/design_scenario.html
